In [1]:
# Description
# Simple Parallel Template to learn Atari agent

In [1]:
class Worker(object):
    def __init__(self, genome, config):
        self.genome = genome
        self.config = config
        
    def work(self):
        
        def preprocess(observation):
            observation = cv2.cvtColor(cv2.resize(observation, (84, 110)), cv2.COLOR_BGR2GRAY)
            observation = observation[26:110,:]
            ret, observation = cv2.threshold(observation,1,255,cv2.THRESH_BINARY)
            #return np.reshape(observation,(84,84,1))
            return np.reshape(observation,(84,84))
    
        def old_preprocess(ob):
            ob = ob[0:190,:,:]
            inx = int(ob.shape[0]/8)
            iny = int(ob.shape[1]/8)
            ob = cv2.resize(ob, (inx, iny))
            ob = cv2.cvtColor(ob, cv2.COLOR_BGR2GRAY)
            return np.reshape(ob, (inx,iny))
        
        # for example: 'SpaceInvaders-v0' or 'Pitfall-v0'
        self.env = gym.make('SpaceInvaders-v0')
        self.env.seed(0)
        self.env.reset()
        
        ac = self.env.action_space.sample()

        #inx, iny, inc = self.env.observation_space.shape
        
        ob, _, _, _ = self.env.step(ac)

        #inx = int(ob.shape[0]/8)
        #iny = int(ob.shape[1]/8)
        done = False

        net = neat.nn.recurrent.RecurrentNetwork.create(self.genome, self.config)
        
        current_max_fitness = 0
        fitness_current = 0
        frame = 0
        counter = 0
        
        #pitfall
        #equOrPosCounterValue = 500
        #posCounterValue = 1000
        finallyCounter = 5000

        while not done:
            
            frame += 1
            
            #pitfall: ob = ob[35:185,7:,:]
            ob = old_preprocess(ob)

            imgarray = np.ndarray.flatten(ob)

            nnOutput = net.activate(imgarray)
            
            # pitfall:
            nnOutput[0] = 0.0
            #nnOutput[1] = 0
            #nnOutput[2] = 0
            #nnOutput[5] = 0
            
            numerical_input = nnOutput.index(max(nnOutput))
            ob, rew, done, info = self.env.step(numerical_input)
            
            #print('action: '+str(self.env.get_action_meanings()[numerical_input])+' rew: '+str(rew)+' done: '+str(done)+' info: '+str(info))
            
            fitness_current += rew
            
            #self.genome.fitness = fitness_current
            
            #print('frame: '+str(frame)+' fitness_current: '+str(fitness_current)+' current_max: '+str(current_max_fitness))
            
            if fitness_current > current_max_fitness:
                current_max_fitness = fitness_current
                counter = 0
            else:
                counter += 1
                
            #pitfall    
            #onlyNegativ = (fitness_current < 0)
            #onlyZero = (counter>=posCounterValue and fitness_current==0)
            finalReached = (counter>=finallyCounter)
            
            #print('final reached: counter >= finalCounter: '+str(counter)+' >= '+str(finallyCounter))
                
            if done or finalReached:
                done = True
                once = True
                #print('this line has to be reached at least once?')
                print('fitness: '+str(fitness_current)+' counter: '+str(counter))
                
        return fitness_current

In [2]:
def eval_genomes(genome, config):
    worky = Worker(genome, config)
    return worky.work()
        

In [3]:
import gym
import numpy as np
import cv2
import neat
import pickle
import random

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# for example: 'SpaceInvaders-v0' or 'Pitfall-v0'
#env = gym.make('SpaceInvaders-v0')
#env.seed(0)

random.seed(0)

imgarray = []
resume = False
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'data/config-space')
p = neat.Population(config)
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
#p.add_reporter(neat.Checkpointer(1))

In [4]:
pe = neat.ParallelEvaluator(8, eval_genomes)

winner = p.run(pe.evaluate)

with open('winner.pkl', 'wb') as output:
    pickle.dump(winner, output, 1)


 ****** Running generation 0 ****** 

fitness: 0.0 counter: 504
fitness: 0.0 counter: 504
fitness: 0.0 counter: 504
fitness: 120.0 counter: 42
fitness: 395.0 counter: 29
fitness: 5.0 counter: 513
fitness: 310.0 counter: 70
fitness: 285.0 counter: 27
fitness: 15.0 counter: 208
fitness: 115.0 counter: 112
fitness: 195.0 counter: 95
fitness: 45.0 counter: 145
fitness: 285.0 counter: 27
fitness: 285.0 counter: 27
fitness: 155.0 counter: 27
fitness: 320.0 counter: 46
fitness: 80.0 counter: 67
fitness: 285.0 counter: 27
fitness: 285.0 counter: 27
fitness: 0.0 counter: 922
Population's average fitness: 159.00000 stdev: 132.92479
Best fitness: 395.00000 - size: (6, 1383) - species 2 - id 3
Average adjusted fitness: 0.462
Mean genetic distance 2.337, standard deviation 0.563
Population of 20 members in 4 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     4    310.0    0.515     0
     2    0     9    395.0    0.366     0
     3    0    

Process ForkPoolWorker-1:
Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Process ForkPoolWorker-4:
  File "/home/nightcrawler/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-6:
Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/home/nightcrawler/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/nightcrawler/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/nightcrawler/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nightcrawler/anaconda3/lib/python3.7/multiprocessing/process.py", lin

  File "/home/nightcrawler/anaconda3/lib/python3.7/site-packages/atari_py/ale_python_interface.py", line 152, in act
    return ale_lib.act(self.obj, int(action))
KeyboardInterrupt
  File "/home/nightcrawler/anaconda3/lib/python3.7/site-packages/atari_py/ale_python_interface.py", line 152, in act
    return ale_lib.act(self.obj, int(action))
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt


KeyboardInterrupt: 